## Predicting the total emission of substances in the future

**Metric: air_total_emission_kg and substance_name**  

# Import libraries

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Import dataset

In [17]:
emision_data = pd.read_csv("./cleaned join tables/Cleaned_join_em_sub_data.csv")

In [18]:
# emision_data = pd.read_csv("./Cleaned_transfers.csv")

In [19]:
emision_data.head()

,report_id,report_year,facility_id,jurisdiction_code,jurisdiction_facility_id,facility_name,registered_business_name,primary_anzsic_class_code,primary_anzsic_class_name,substance_id,...,cat2a,cat2b,cat3,voc,c1_substance_use_threshold_kg,emissions_to_air,emissions_to_water,emissions_to_land,fact_sheet_url,first_reporting_year
0,525732,2020,516654,SA,SA0152,YALDARA ESTATE,1847 WINERY (SA) PTY LTD,1214,Wine and Other Alcoholic Beverage Manufacturing,37,...,False,False,False,True,10000.0,True,True,True,http://www.npi.gov.au/substances/ethanol/index...,2001
1,525732,2020,516654,SA,SA0152,YALDARA ESTATE,1847 WINERY (SA) PTY LTD,1214,Wine and Other Alcoholic Beverage Manufacturing,85,...,True,True,False,False,25000.0,True,False,False,http://www.npi.gov.au/substances/volatile-orga...,2001
2,524380,2020,545702,WA,WA0685,3 Oceans Wine Company,3 OCEANS WINE COMPANY PTY LTD,1214,Wine and Other Alcoholic Beverage Manufacturing,37,...,False,False,False,True,10000.0,True,True,True,http://www.npi.gov.au/substances/ethanol/index...,2001
3,524380,2020,545702,WA,WA0685,3 Oceans Wine Company,3 OCEANS WINE COMPANY PTY LTD,1214,Wine and Other Alcoholic Beverage Manufacturing,85,...,True,True,False,False,25000.0,True,False,False,http://www.npi.gov.au/substances/volatile-orga...,2001
4,526218,2020,658102,NSW,1538,4 Pines Brewing Brookvale,4 PINES BREWING COMPANY WHOLESALE PTY LTD,1212,Beer Manufacturing,37,...,False,False,False,True,10000.0,True,True,True,http://www.npi.gov.au/substances/ethanol/index...,2001


In [20]:
emision_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830263 entries, 0 to 830262
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   report_id                      830263 non-null  int64  
 1   report_year                    830263 non-null  int64  
 2   facility_id                    830263 non-null  int64  
 3   jurisdiction_code              830263 non-null  object 
 4   jurisdiction_facility_id       830263 non-null  object 
 5   facility_name                  830263 non-null  object 
 6   registered_business_name       824021 non-null  object 
 7   primary_anzsic_class_code      830263 non-null  int64  
 8   primary_anzsic_class_name      830263 non-null  object 
 9   substance_id                   830263 non-null  int64  
 10  substance_sort_order           830263 non-null  int64  
 11  substance_name_x               830263 non-null  object 
 12  air_point_emission_kg         

In [21]:
#convert the report_year column to datetime
emision_data['report_year'] = pd.to_datetime(emision_data['report_year'],format='%Y')


# EDA

In [22]:
## Find the mean and count of air_point_emission_kg column group by report_year
emision_data .groupby("report_year")["air_point_emission_kg"].agg(["count", "mean"]).sort_values("mean", ascending=False) # sort by highest % of 1s

,count,mean
report_year,,
1998-01-01,6242,300398.605401
2000-01-01,20001,226853.402286
1999-01-01,15539,221945.898600
2001-01-01,29251,202723.366231
2003-01-01,33788,161761.235768
2004-01-01,34117,156438.234848
2002-01-01,32879,154292.354015
2005-01-01,35148,146664.245631
2006-01-01,35453,140742.400989


In [23]:
## Fill the NaN values with mean of the corresponding year
emision_data['air_point_emission_kg'].fillna(emision_data.groupby('report_year')['air_point_emission_kg'].transform("mean"),
                                             inplace=True)

In [24]:
## Finally get the mean and count of air_total_emission_kg column group by report_year
emision_data .groupby("report_year")["air_total_emission_kg"].agg(["mean"]).to_csv('./time_series_air_total_emission.csv', index=True)